En primer lugar procedo a importar las bibliotecas para poder trabajar con spark, asi como realizar la inicialisacion del mismo

In [ ]:

#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.2//spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [96.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http:

'/content/spark-3.2.2-bin-hadoop3.2'

Monto mis unidades de Drive ya que en estas se encuentran los arhivos csv con los que voy a trabajar

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")
%cd "/content/gdrive/Shareddrives/Reto AI SEPTIMO"

Mounted at /content/gdrive
/content/gdrive/Shareddrives/Reto AI SEPTIMO


Verifico que Pyspark este correctamente instalado y este sea funcional

In [ ]:
#Verificar la funcionalidad de Pyspark 
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType,FloatType 
spark_session = SparkSession.builder.appName('Modulo1_Analisis').config("spark.driver.memory", "16g").getOrCreate()
spark_session


Procedo a abrir mi csv detro de mi mi spark session, siendo que el csv que voy a utilizar es un csv ya limpio que obtuvimos para nuestro reto de bloque.

In [ ]:
df_spark = spark_session.read.csv('telcom_numeric.csv')
df_spark

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string]

Si bien no deberia ya porque haber datos NaN, procedo a realizar un drop nan para estar completamente seguro de esto, ya que en algunos casos esto puede llegar a causar problemas con el desarrollo del modelo

In [ ]:

df_spark = df_spark.na.drop("all")



Hago un show para revisar que mi archivo csv esta siendo correctamente leido, in embargo como poemos observar en este caso, n esta tomando os titulos de las columnas como headers, si no que los ingresa dentro de las rows del dataframe, por lo que esto tendra que ser correguido ya que en caso de que no se corrija, no se podra realizar la modelacion de este modelo.


In [ ]:
df_spark.show()

+----+------+-----------+------------+------------------+------------------+--------------------+------------------+-------------+--------------+-----------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------------+-------------+-------------+----------+------------+--------------------+---------------+------+--------------------+-----------------+
| _c0|   _c1|        _c2|         _c3|               _c4|               _c5|                 _c6|               _c7|          _c8|           _c9|       _c10|              _c11|                _c12|                _c13|                _c14|                _c15|              _c16|                _c17|             _c18|         _c19|         _c20|      _c21|        _c22|                _c23|           _c24|  _c25|                _c26|             _c27|
+----+------+-----------+------------+------------------+------------------+----------------

En esta opcion vuelvo a leer mi archivo csv, solo que en este caso procedo a indicar que se debe tomar el primer renglon de mi archivo como los header del dataframe

In [ ]:
#La opción option('header','true')
df_spark_col  = spark_session.read.option('header', 'true').csv('telcom_numeric.csv')
df_spark_col

DataFrame[_c0: string, TARGET: string, YEAR_JOINED: string, CURRENT_YEAR: string, BILL_AMOUNT: string, PAID_AMOUNT: string, PAYMENT_TRANSACTIONS: string, PARTY_REV: string, PREPAID_LINES: string, POSTPAID_LINES: string, OTHER_LINES: string, LINE_REV: string, MOUS_TO_LOCAL_MOBILES: string, MOUS_FROM_LOCAL_MOBILES: string, MOUS_TO_LOCAL_LANDLINES: string, MOUS_FROM_LOCAL_LANDLINES: string, MOUS_TO_INT_NUMBER: string, MOUS_FROM_INT_NUMBER: string, DATA_IN_BNDL: string, DATA_OUT_BNDL: string, DATA_USG_PAYG: string, COMPLAINTS: string, Years_stayed: string, PTY_PROFILE_SUB_TYPE: string, PARTY_GENDER_CD: string, STATUS: string, SOCIO_ECONOMIC_SEGMENT: string, PARTY_NATIONALITY: string]

In [ ]:
df_spark_col.show()

+---+------+-----------+------------+------------------+------------------+--------------------+------------------+-------------+--------------+-----------+------------------+---------------------+-----------------------+-----------------------+-------------------------+------------------+--------------------+-----------------+-------------+-------------+----------+------------+--------------------+---------------+------+----------------------+-----------------+
|_c0|TARGET|YEAR_JOINED|CURRENT_YEAR|       BILL_AMOUNT|       PAID_AMOUNT|PAYMENT_TRANSACTIONS|         PARTY_REV|PREPAID_LINES|POSTPAID_LINES|OTHER_LINES|          LINE_REV|MOUS_TO_LOCAL_MOBILES|MOUS_FROM_LOCAL_MOBILES|MOUS_TO_LOCAL_LANDLINES|MOUS_FROM_LOCAL_LANDLINES|MOUS_TO_INT_NUMBER|MOUS_FROM_INT_NUMBER|     DATA_IN_BNDL|DATA_OUT_BNDL|DATA_USG_PAYG|COMPLAINTS|Years_stayed|PTY_PROFILE_SUB_TYPE|PARTY_GENDER_CD|STATUS|SOCIO_ECONOMIC_SEGMENT|PARTY_NATIONALITY|
+---+------+-----------+------------+------------------+----------

Ahora si bien es cierto que ya reparamos los headers le dataframe, podemos obsevar que las columnas no tienen definido el correcto tipo de dato, siendo que todas fueron identificadas como si fueran de tipo string

In [ ]:
df_spark_col.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- TARGET: string (nullable = true)
 |-- YEAR_JOINED: string (nullable = true)
 |-- CURRENT_YEAR: string (nullable = true)
 |-- BILL_AMOUNT: string (nullable = true)
 |-- PAID_AMOUNT: string (nullable = true)
 |-- PAYMENT_TRANSACTIONS: string (nullable = true)
 |-- PARTY_REV: string (nullable = true)
 |-- PREPAID_LINES: string (nullable = true)
 |-- POSTPAID_LINES: string (nullable = true)
 |-- OTHER_LINES: string (nullable = true)
 |-- LINE_REV: string (nullable = true)
 |-- MOUS_TO_LOCAL_MOBILES: string (nullable = true)
 |-- MOUS_FROM_LOCAL_MOBILES: string (nullable = true)
 |-- MOUS_TO_LOCAL_LANDLINES: string (nullable = true)
 |-- MOUS_FROM_LOCAL_LANDLINES: string (nullable = true)
 |-- MOUS_TO_INT_NUMBER: string (nullable = true)
 |-- MOUS_FROM_INT_NUMBER: string (nullable = true)
 |-- DATA_IN_BNDL: string (nullable = true)
 |-- DATA_OUT_BNDL: string (nullable = true)
 |-- DATA_USG_PAYG: string (nullable = true)
 |-- COMPLAINTS: string (n

En este ciclo for se procede a realizar la trnsformacion del tipo de variable sus respectivas columnas

In [ ]:
for column in df_spark_col.columns:
  if column not in ["_c0", "TARGET", "YEAR_JOINED", "CURRENT_YEAR", "PAYMENT_TRANSACTIONS", "PREPAID_LINES", "POSTPAID_LINES", "OTHER_LINES", "COMPLAINTS", "PARTY_GENDER_CD", "STATUS", "SOCIO_ECONOMIC_SEGMENT", "PARTY_NATIONALITY"]:
    df_spark_col = df_spark_col.withColumn(column, col(column).cast(FloatType()))
  else:
    df_spark_col = df_spark_col.withColumn(column, col(column).cast(IntegerType()))


df_spark_col.show()

+---+------+-----------+------------+-----------+-----------+--------------------+---------+-------------+--------------+-----------+---------+---------------------+-----------------------+-----------------------+-------------------------+------------------+--------------------+------------+-------------+-------------+----------+------------+--------------------+---------------+------+----------------------+-----------------+
|_c0|TARGET|YEAR_JOINED|CURRENT_YEAR|BILL_AMOUNT|PAID_AMOUNT|PAYMENT_TRANSACTIONS|PARTY_REV|PREPAID_LINES|POSTPAID_LINES|OTHER_LINES| LINE_REV|MOUS_TO_LOCAL_MOBILES|MOUS_FROM_LOCAL_MOBILES|MOUS_TO_LOCAL_LANDLINES|MOUS_FROM_LOCAL_LANDLINES|MOUS_TO_INT_NUMBER|MOUS_FROM_INT_NUMBER|DATA_IN_BNDL|DATA_OUT_BNDL|DATA_USG_PAYG|COMPLAINTS|Years_stayed|PTY_PROFILE_SUB_TYPE|PARTY_GENDER_CD|STATUS|SOCIO_ECONOMIC_SEGMENT|PARTY_NATIONALITY|
+---+------+-----------+------------+-----------+-----------+--------------------+---------+-------------+--------------+-----------+-------

Procedemos comprobar que el cambio se haya realizado de manera correcta, y como se puede ver si fue asi

In [ ]:
df_spark_col.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- YEAR_JOINED: integer (nullable = true)
 |-- CURRENT_YEAR: integer (nullable = true)
 |-- BILL_AMOUNT: float (nullable = true)
 |-- PAID_AMOUNT: float (nullable = true)
 |-- PAYMENT_TRANSACTIONS: integer (nullable = true)
 |-- PARTY_REV: float (nullable = true)
 |-- PREPAID_LINES: integer (nullable = true)
 |-- POSTPAID_LINES: integer (nullable = true)
 |-- OTHER_LINES: integer (nullable = true)
 |-- LINE_REV: float (nullable = true)
 |-- MOUS_TO_LOCAL_MOBILES: float (nullable = true)
 |-- MOUS_FROM_LOCAL_MOBILES: float (nullable = true)
 |-- MOUS_TO_LOCAL_LANDLINES: float (nullable = true)
 |-- MOUS_FROM_LOCAL_LANDLINES: float (nullable = true)
 |-- MOUS_TO_INT_NUMBER: float (nullable = true)
 |-- MOUS_FROM_INT_NUMBER: float (nullable = true)
 |-- DATA_IN_BNDL: float (nullable = true)
 |-- DATA_OUT_BNDL: float (nullable = true)
 |-- DATA_USG_PAYG: float (nullable = true)
 |-- COMPLAINTS: integer (nulla

En esta parte procedemos a convertir ahora nuestro dataframe de pyspark a un tipo de dataframe RDD, para con esto poder hacer uso de la libreria de Milib para realizar las predicciones y modelos que se requieren

In [ ]:
df_RDD = df_spark_col.rdd.map(lambda x: LabeledPoint(x[1], x[2:]))
df_RDD.take(10)

[LabeledPoint(0.0, [1994.0,2019.0,931.2089233398438,812.1749877929688,1.0,5968.7001953125,2.0,6.0,2.0,945.0399780273438,1004.0700073242188,35.849998474121094,34.01499938964844,72.07499694824219,141.83999633789062,56.1150016784668,11944.0791015625,0.0,0.0,0.0,25.0,2.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,431.0826110839844,486.5,1.0,6245.1416015625,6.0,4.0,2.0,493.81500244140625,159.0500030517578,10.595000267028809,7.715000152587891,11.75,5.110000133514404,0.0,9903.158203125,0.0,0.0,0.0,25.0,1.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,50.61964416503906,52.814998626708984,1.0,1666.48828125,2.0,3.0,1.0,50.29999923706055,0.0,0.0,0.0,0.0,0.0,0.0,0.1025390625,0.0,0.0,0.0,25.0,2.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,399.71002197265625,422.2349853515625,1.0,2522.00830078125,3.0,4.0,3.0,406.586669921875,288.80499267578125,158.5,2.6700000762939453,15.96500015258789,0.0,0.0,3600.322265625,0.0,0.0,0.0,25.0,1.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.

Procedo a dividir mi dataframe en test y train para poder realizar comprobar el modelo que se va a realizar

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_RDD.randomSplit([0.7,0.3])

Procedo a realizar un modelo de clasificacion de mi dataframe, con ayuda de la funcion de SVMWithSGD, de Milib

In [ ]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

model = SVMWithSGD.train(trainingData, iterations=100)

In [ ]:
model

(weights=[-44.53749111219793,-47.26365460081127,-98.29989843470761,-103.21815913077384,-0.04100622113519156,-45.68691717955698,1.2279195127018092,2.3820131357304093,0.19305310637331913,-100.61329255561192,-157.15725389046872,-46.316341186995956,-7.3011866730842865,-10.88948149714478,-2.3366124631036618,-2.387901525567747,-437.21389883625346,-0.006212859527732977,-72.11757429761516,-0.0030313190407505915,-2.7261634886136936,-0.04927039212927896,0.0029229208110651585,0.22143056052472262,0.0919558913610708,-10.463528286777473], intercept=0.0)

Procedemo a calcular el error que existe en nuestro modelo para saber la fiabilidad del mismo

In [ ]:
labelsAndPreds = testData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.05278872531691056
